In [5]:
pwd()


"/Users/janzubalewicz/Documents/My/WWA/mgr/nauka/awid/MyProject"

In [ ]:
exit()


In [1]:
cd("/Users/janzubalewicz/Documents/My/WWA/mgr/nauka/awid/MyProject")
using Pkg
Pkg.activate(".")

  Activating project at `~/Documents/My/WWA/mgr/nauka/awid/MyProject`


In [2]:
using MyProject
using MyProject.MyCNN
mycnn = MyProject.MyCNN
using MyProject.MyAD
myad = MyProject.MyAD
using JLD2
using Random


X_train = load("data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("data/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("data/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("data/imdb_dataset_prepared.jld2", "vocab")

embedding_dim, vocab_size = size(embeddings)

println("X_train shape: ", size(X_train))
println("y_train shape: ", size(y_train))
println("X_test shape: ", size(X_test))
println("y_test shape: ", size(y_test))
println("Embedding dim: ", size(embeddings))
println("Vocab size: ", length(vocab))

[ Info: Precompiling MyProject [be156bc0-6739-40ee-bb06-19190d46f194] (cache misses: include_dependency fsize change (2), mismatched flags (2))


X_train shape: (130, 40000)
y_train shape: (1, 40000)
X_test shape: (130, 10000)
y_test shape: (1, 10000)
Embedding dim: (50, 12849)
Vocab size: 12849


In [3]:
function to_onehot(y, n_classes)
    y_oh = zeros(Float32, n_classes, length(y))
    for (i, label) in enumerate(y)
        y_oh[label, i] = 1.0
    end
    return y_oh
end

n_classes = 2

y_train_int = Int.(vec(y_train)) .+ 1
y_test_int = Int.(vec(y_test)) .+ 1

y_train_oh = to_onehot(y_train_int, n_classes)
y_test_oh = to_onehot(y_test_int, n_classes)

2×10000 Matrix{Float32}:
 1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  …  0.0  0.0  1.0  1.0  1.0  1.0  1.0
 0.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0     1.0  1.0  0.0  0.0  0.0  0.0  0.0

In [4]:
embedding_layer = Embedding(embeddings)
n_filters = 8
filter_sizes = [3]
model = CNN(
    embedding=embedding_layer,
    n_filters=n_filters,
    filter_sizes=filter_sizes,
    n_classes=n_classes,
    seq_len=size(X_train, 2)  # or whatever your actual input length is
)

CNN(Embedding(Float32[0.90951 -0.58014 … 0.014323 0.0; -0.20702 -1.1316 … -0.74624 0.0; … ; -0.60396 0.1722 … -0.079873 0.0; -0.31136 1.2622 … -0.89164 0.0]), Conv2D[Conv2D(Float32[0.16772954 -0.06669945 … 0.034587335 0.030594831; 0.11891296 0.16593625 … -0.0004780615 -0.19930357; 0.06428459 0.10529229 … -0.012297262 -0.07825297;;;; 0.03883656 -0.13476834 … 0.19175966 -0.119388096; -0.08917357 -0.026040507 … -0.018399162 -0.012587911; -0.00570072 0.04816953 … -0.042767406 0.048305232;;;; 0.04464904 0.16295858 … 0.14485067 0.02354971; 0.13588227 0.13434424 … 0.0052043144 -0.032598704; 0.114609264 0.034051444 … 0.06469068 0.060418252;;;; -0.18281847 0.05067956 … 0.14040409 0.09622225; 0.13050775 0.08306196 … -0.20279896 -0.015844224; -0.22552426 0.036942028 … 0.07280967 -0.020347634;;;; -0.13252896 -0.008793123 … 0.11027092 -0.05129517; -0.10245298 -0.20331526 … -0.109872155 -0.14208245; 0.16581386 -0.30127457 … -0.03246832 -0.13449125;;;; 0.23396331 -0.21119258 … -0.03890189 -0.25878218

In [5]:
params = parameters(model)
adam_state = mycnn.init_adam(params)
batch_size = 64
epochs = 3
lr = 1e-3

for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_batches = 0

    for i in 1:batch_size:size(X_train, 1)
        idx = i:min(i+batch_size-1, size(X_train, 1))
        x_batch = X_train[idx, :]
        y_batch = y_train_oh[:, idx]

        mycnn.zero_grads!(model)
        y_pred = mycnn.forward(model, x_batch)
        loss = mycnn.my_loss(y_pred, y_batch)
        grads = mycnn.backward(model, x_batch, y_batch, y_pred)
        mycnn.adam_step!(params, grads, adam_state, lr)
        acc = mycnn.my_accuracy(y_pred, y_batch)

        total_loss += loss
        total_acc += acc
        num_batches += 1
    end

    println("Epoch $epoch: loss = $(total_loss/num_batches), accuracy = $(total_acc/num_batches)")
end

┌ Info: conv2d input
│   size_x = (64, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (64, 8, 39998, 1)
┌ Info: conv2d input
│   size_x = (64, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (64, 8, 39998, 1)
┌ Info: conv2d input
│   size_x = (2, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (2, 8, 39998, 1)


Epoch 1: loss = 0.2630158265431722, accuracy = 0.53125


┌ Info: conv2d input
│   size_x = (64, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (64, 8, 39998, 1)
┌ Info: conv2d input
│   size_x = (64, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (64, 8, 39998, 1)
┌ Info: conv2d input
│   size_x = (2, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (2, 8, 39998, 1)


Epoch 2: loss = 0.26280590891838074, accuracy = 0.53125


┌ Info: conv2d input
│   size_x = (64, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (64, 8, 39998, 1)
┌ Info: conv2d input
│   size_x = (64, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (64, 8, 39998, 1)
┌ Info: conv2d input
│   size_x = (2, 1, 40000, 50)
└   size_filters = (3, 50, 1, 8)
┌ Info: conv2d output reshaped
└   size_output = (2, 8, 39998, 1)


Epoch 3: loss = 0.2625993291536967, accuracy = 0.53125
